In [1]:
import json
from time import time
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import requests
import pandas as pd
from numpy import mean
import pandas as pd
import boto3
import os
import time
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from util.preprocess import *
import trp
import pickle
# load saved model
with open('model_pkl2' , 'rb') as f:
    model = pickle.load(f)

/home/ubuntu/.local/lib/python3.10/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.24.1 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintaina

In [2]:
# Use a service account
cred = credentials.Certificate('medisca-firebase-adminsdk-6jxve-caa474f363.json')
firebase_admin.initialize_app(cred)

In [3]:
def extract_text(textractJobId,response):
    pages = []

    time.sleep(5)
    textract = boto3.client('textract')
    response = textract.get_document_analysis(JobId=textractJobId)

    pages.append(response)

    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']

    while(nextToken):
        time.sleep(5)
        textract = boto3.client('textract')
        response = textract.get_document_analysis(JobId=textractJobId, NextToken=nextToken)

        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']
    
    return pages 

In [4]:
def extractTextract(textractObjectName):
    textract = boto3.client('textract')
    response = textract.start_document_analysis(
        DocumentLocation={
            'S3Object': {
            'Bucket': 'mediscan-aws-hackathon',
            'Name': textractObjectName,

        }
            },
        FeatureTypes=[
            'TABLES',
        ]
        )

    textractJobId = response["JobId"]
    print('job id is: ',textractJobId)
    time.sleep(15)
    response = textract.get_document_analysis(JobId=textractJobId)
    status = response["JobStatus"]

    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = textract.get_document_analysis(JobId=textractJobId)
        status = response["JobStatus"]
        print("Textract Job status: {}".format(status))
    
    pages=extract_text(textractJobId,response)
    doc = trp.Document(pages)
    return doc

In [5]:
def extractMedical(doc):
    maxLength = 10000
    comprehendResponse = []
    comprehend_medical_client = boto3.client(service_name='comprehendmedical', region_name='ap-southeast-2')
    for page in doc.pages:
        pageText = page.text
    
        for i in range(0, len(pageText), maxLength):
            response = comprehend_medical_client.detect_entities_v2(Text=pageText[0+i:maxLength+i])
            comprehendResponse.append(response)
        patient_string = ""
        
    #df_cm=extractMC_v2(comprehendResponse[0])
    return comprehendResponse

In [6]:
db = firestore.client()

reports_ref = db.collection('reports')
reports = reports_ref.stream()

In [38]:
from apt import ProblemResolver


def getOutput(fileName):
    textractObjectName = os.path.join('public', fileName)
    print("EHR file to be processed is at ", textractObjectName)
    textract = boto3.client('textract')
    doc=extractTextract(textractObjectName)
    #extract data from comprehend medical
    comprehendResponse=extractMedical(doc)
    df_cm=extractMC_v2(comprehendResponse[0])
    #Organize the extracted json file into dataframe
    mclist, df_cm2=retrieve_mcList(df_cm, nFeature=20,threshold=0.9)
    df_cm2=df_mc_generator_slim(df_cm2)
    df_cm2_aug = df_cm2.drop('ID', axis=1)
    sample_pred = model.predict(df_cm2_aug)
    prob = model.predict_proba(df_cm2_aug)
    p = [int(sample_pred),prob]
    print(p, df_cm2.to_dict())
    return p, df_cm2.to_dict()

        

In [39]:
pred , values = getOutput('sample_report_1.pdf')

EHR file to be processed is at  public/sample_report_1.pdf
job id is:  cd24232955fcd84b029529955859c7711b003f5b392b4db1f2e6b22e68bfcd61
[0, array([[0.69589794, 0.30410203]], dtype=float32)] {'ID': {0: 0}, 'nontender': {0: 0.9877710342407227}, 'foreign body': {0: 0}, 'edema': {0: 0}, 'alert': {0: 0}, 'murmur': {0: 0}, 'chest pain': {0: 0}, 'vomiting': {0: 0.9983489513397217}, 'hiatal hernia': {0: 0}, 'distress': {0: 0}, 'hemostasis': {0: 0}, 'carpal tunnel syndrome': {0: 0}, 'endometriosis': {0: 0}, 'weakness': {0: 0}, 'pain': {0: 0}, 'mass': {0: 0.9822571873664856}, 'inflammation': {0: 0}, 'polyp': {0: 0}, 'bleeding': {0: 0}, 'hypertension': {0: 0.6807340383529663}, 'supple': {0: 0.9988003969192505}, 'fever': {0: 0}, 'stenosis': {0: 0}, 'wound': {0: 0}, 'cyanosis': {0: 0}, 'infection': {0: 0}, 'erythema': {0: 0}, 'normocephalic': {0: 0.949869692325592}, 'fracture': {0: 0}, 'lesion': {0: 0.9984696507453918}, 'ulceration': {0: 0}, 'nausea': {0: 0}, 'cough': {0: 0}, 'tumor': {0: 0}, 'soft

/home/ubuntu/mediscan/aws/util/preprocess.py:71: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_mc = pd.DataFrame({'MEDICAL_CONDITION': pd.Series(medical_conditions), 'Score':pd.Series(scores),'Trait':pd.Series(traits)})


In [24]:
keys_list = []
values_list = []
for key,value in values.items():
    if value[0] > 0:
        keys_list.append(key)
        values_list.append(values)


0.69589794

In [47]:

data = {
    'score'  : str(round(pred[1].max(),3)),
    'status' : 'success'
}
db = firestore.client()
doc_ref = db.collection('reports')
doc_ref.document('one').set(data)

update_time {
  seconds: 1666441833
  nanos: 73036000
}

In [40]:
pred

[0, array([[0.69589794, 0.30410203]], dtype=float32)]

In [33]:
model.predict_proba(df_cm2_aug)

NameError: name 'df_cm2_aug' is not defined